In [101]:
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

# 데이터 세트를 읽어들인다.
df = pd.read_csv("/content/whisky.csv", sep=',')

# 결측치 제거
df = df.dropna(subset=['Honey'])

# 정수형 변환
df['Honey'] = df['Honey'].astype(int)

df_encoded = pd.get_dummies(df, columns=['Distillery'], drop_first=True)

# 입력 데이터 X : 'Honey' 컬럼 제외한 모든 컬럼 사용
X = df_encoded.drop(columns=['Honey'])

y = df_encoded['Honey']

y=keras.utils.to_categorical(y)

print()

mms=MinMaxScaler()
mms.fit(X)
print(X.head())
X_mms=mms.transform(X)
X = pd.DataFrame(X_mms, columns=X.columns, index=list(X.index.values))


print(X.head())
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,shuffle=True)
print(df.Honey)


# 케라스 모델을 생성한다.
input_dim = X.shape[1]
model = keras.models.Sequential()
model.add(keras.layers.Dense(64, activation='tanh', input_shape=(input_dim,)))
model.add(keras.layers.Dense(64, activation='tanh'))
model.add(keras.layers.Dense(64, activation='tanh'))
model.add(keras.layers.Dense(32, activation='tanh'))
model.add(keras.layers.Dense(5,activation='softmax'))

# 케라스 모델을 컴파일한다.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# 20회 이상 결과가 향상되지 않으면 자동으로 중단되게끔 합니다.
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

# 모델의 이름을 정합니다.
modelpath='/content/whisky_modelr.keras'

# 최적화 모델을 업데이트하고 저장합니다.
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

# 실행 관련 설정을 하는 부분입니다. 전체의 20%를 검증셋으로 설정합니다.
history = model.fit(X_train, y_train, validation_split=0.25, epochs=2000, batch_size=16, callbacks=[early_stopping_callback, checkpointer])

# 테스트 결과를 출력합니다.
score=model.evaluate(X_test, y_test)
print('Test accuracy:', score[1])



   RowID  Body  Sweetness  Smoky  Medicinal  Tobacco  Spicy  Winey  Nutty  \
0    1.0   2.0        2.0    2.0        0.0      0.0    1.0    2.0    2.0   
1    2.0   3.0        3.0    1.0        0.0      0.0    3.0    2.0    2.0   
2    3.0   1.0        3.0    2.0        0.0      0.0    0.0    0.0    2.0   
3    4.0   4.0        1.0    4.0        4.0      0.0    2.0    0.0    1.0   
4    5.0   2.0        2.0    2.0        0.0      0.0    1.0    1.0    2.0   

   Malty  ...  Distillery_Strathmill  Distillery_Talisker  Distillery_Tamdhu  \
0    2.0  ...                  False                False              False   
1    3.0  ...                  False                False              False   
2    2.0  ...                  False                False              False   
3    2.0  ...                  False                False              False   
4    3.0  ...                  False                False              False   

   Distillery_Tamnavulin  Distillery_Teaninich  Distill

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/3 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.1250 - loss: 1.6009
Epoch 1: val_loss improved from inf to 1.57071, saving model to /content/whisky_modelr.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 194ms/step - accuracy: 0.1102 - loss: 1.5844 - val_accuracy: 0.1333 - val_loss: 1.5707
Epoch 2/2000
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.2500 - loss: 1.3996
Epoch 2: val_loss improved from 1.57071 to 1.47926, saving model to /content/whisky_modelr.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.2839 - loss: 1.3851 - val_accuracy: 0.4000 - val_loss: 1.4793
Epoch 3/2000
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - accuracy: 0.5625 - loss: 1.2341
Epoch 3: val_loss improved from 1.47926 to 1.40950, saving model to /content/whisky_modelr.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5701 - loss: 1.2364 - val_accuracy: 0.3333 - val_loss: 1.4095
Epoch 4/2000
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6875 - loss: 1.0784
Epoch 4: val_loss improved from 1.40950 to 1